In [2]:
!pip install keras-tuner --upgrade


   ---------------------------------------- 0/2 [kt-legacy]
   ---------------------------------------- 0/2 [kt-legacy]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   --------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, Input
import keras_tuner as kt  # ✅ correct


In [4]:
import sys
print(sys.executable)


D:\Jupyter\venv\Scripts\python.exe


In [5]:
# Load and preprocess dataset (MNIST)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
x_val, y_val = x_train[-10000:], y_train[-10000:]
x_train, y_train = x_train[:-10000], y_train[:-10000]


11490434/11490434 [==============================] - 3s 0us/step


In [6]:
# =========================
# STEP 1: Search Space Setup
# =========================
search_space = {
    'filters': [32, 64, 96, 128],
    'kernel_size': [3, 5],
    'dense_units': [64, 128, 192, 256],
    'dropout': [0.2, 0.3, 0.4, 0.5],
    'learning_rate': [1e-2, 1e-3, 1e-4]
}

In [7]:

# ==========================================
# STEP 2: Wrap Training + Evaluation Function
# ==========================================
def build_model(hp):
    model = keras.Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(layers.Conv2D(
        filters=hp.Choice('filters', search_space['filters']),
        kernel_size=hp.Choice('kernel_size', search_space['kernel_size']),
        activation='relu'
    ))
    model.add(layers.MaxPooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(
        units=hp.Choice('dense_units', search_space['dense_units']),
        activation='relu'
    ))
    model.add(layers.Dropout(
        hp.Choice('dropout', search_space['dropout'])
    ))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', search_space['learning_rate'])
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [8]:
# ===================================
# STEP 3: Run the Hyperparameter Tuner
# ===================================
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='my_dir',
    project_name='cnn_tuning_example'
)

In [ ]:

# Start tuning
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Trial 2 Complete [00h 01m 46s]
val_accuracy: 0.9890000224113464

Best val_accuracy So Far: 0.9890000224113464
Total elapsed time: 00h 03m 48s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
32                |32                |filters
5                 |5                 |kernel_size
64                |192               |dense_units
0.3               |0.5               |dropout
0.01              |0.001             |learning_rate

Epoch 1/5
1563/1563 [==============================] - 19s 12ms/step - loss: 0.2144 - accuracy: 0.9361 - val_loss: 0.0767 - val_accuracy: 0.9778
Epoch 2/5
1563/1563 [==============================] - 21s 13ms/step - loss: 0.1350 - accuracy: 0.9613 - val_loss: 0.1051 - val_accuracy: 0.9735
Epoch 3/5
 210/1563 [===>..........................] - ETA: 14s - loss: 0.1332 - accuracy: 0.9579

In [ ]:

# Best model summary (optional)
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()
# Show a summary of top trials
tuner.results_summary()

In [ ]:
# Print accuracy of all trials
print("\nAll Trial Accuracies:")
for trial in tuner.oracle.get_best_trials(num_trials=len(tuner.oracle.trials)):
    val_acc = trial.metrics.get_last_value('val_accuracy')
    print(f"Trial ID: {trial.trial_id} | Validation Accuracy: {val_acc:.4f}")